In [1]:
import glob
import os

import lightning
import optuna
import torch
from lightning import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from terratorch.tasks.tiled_inference import TiledInferenceParameters

from implementation.training.utilities import (
    TrainingTaskHyperparameterTuningConfig,
    create_model,
)
from roofsense.bag3d import BAG3DTileStore
from roofsense.inference.tile import TiledInferenceEngine
from roofsense.training.datamodule import TrainingDataModule
from roofsense.training.task import TrainingTask

C:\Documents\RoofSense\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.5 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
study_name = "optimization"
optim_log_dirpath = os.path.join(r"C:\Documents\RoofSense\logs\3dgeoinfo", study_name)

study = optuna.load_study(
    study_name="optim", storage=f"sqlite:///{optim_log_dirpath}/storage.db"
)

best_params = study.best_params
# Convert parameter format.
for param in ["lab", "tgi"]:
    best_params[f"append_{param}"] = best_params.pop(param)

config = TrainingTaskHyperparameterTuningConfig(
    # Add constant parameters.
    # Encoder
    encoder="tu-resnet18d",
    zero_init_last=True,
    # Loss
    label_smoothing=0.1,
    # Optimizer
    optimizer="AdamW",
    # LR Scheduler
    lr_scheduler="CosineAnnealingLR",
    **best_params,
)

In [3]:
# Quantitative Performance Evaluation
log_dirpath = os.path.join(os.path.dirname(optim_log_dirpath), "test")

torch.set_float32_matmul_precision("high")
torch.backends.cudnn.allow_tf32 = True

lightning.pytorch.seed_everything(0, workers=True)

model = create_model(config)

# Best Step
best_path = os.path.join(
    optim_log_dirpath, f"version_{study.best_trial.number}", "ckpts", "best.ckpt"
)

task: TrainingTask = TrainingTask.load_from_checkpoint(
    checkpoint_path=best_path, model=model
)

datamodule = TrainingDataModule(
    root=r"C:\Documents\RoofSense\roofsense\dataset",
    append_lab=config.append_lab,
    append_tgi=config.append_tgi,
)

trainer = Trainer(
    logger=TensorBoardLogger(
        save_dir=os.path.join(log_dirpath, "quantitative"), name="best"
    ),
    benchmark=True,
)
trainer.test(task, datamodule=datamodule)

# Last Step
for last_path in glob.glob(
    os.path.join(optim_log_dirpath, "validation", "version_*", "ckpts", "last.ckpt")
):
    task: TrainingTask = TrainingTask.load_from_checkpoint(
        checkpoint_path=last_path, model=model
    )

    datamodule = TrainingDataModule(
        root=r"C:\Documents\RoofSense\roofsense\dataset",
        append_lab=config.append_lab,
        append_tgi=config.append_tgi,
    )

    trainer = Trainer(
        logger=TensorBoardLogger(
            save_dir=os.path.join(log_dirpath, "quantitative"), name="last"
        ),
        benchmark=True,
    )
    trainer.test(task, datamodule=datamodule)

Seed set to 0
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/resnet18d.ra2_in1k)
INFO:timm.models._hub:[timm/resnet18d.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Converted input conv conv1.0 pretrained weights from 3 to 10 channel(s)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 6/6 [00:17<00:00,  0.34it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       tst/Accuracy0       │            0.0            │
│       tst/Accuracy1       │    0.9483305215835571     │
│       tst/Accuracy2       │    0.8825523853302002     │
│       tst/Accuracy3       │    0.9835895299911499     │
│       tst/Accuracy4       │     0.920958399772644     │
│       tst/Accuracy5       │    0.5458983778953552     │
│       tst/Accuracy6       │    0.9795466065406799     │
│       tst/Accuracy7       │     0.757390558719635     │
│       tst/Accuracy8       │    0.7807261347770691     │
│       tst/F1Score0        │            0.0            │
│       tst/F1Score1        │    0.9261047840118408     │
│       tst/F1Score2        │    0.9206501841545105     │
│       tst/F1Score3        │    0.9594050049781799     │
│       tst/F1Score4        │    0.9283773303031921     │
│       tst/F1Score5        │    0.5603823661804199     │
│       tst/F1Score6        │     0.931647777557373     │
│       tst/F1Score7        │    0.7900201678276062     │
│       tst/F1Score8        │    0.7189778685569763     │
│     tst/JaccardIndex0     │            0.0            │
│     tst/JaccardIndex1     │    0.8623791337013245     │
│     tst/JaccardIndex2     │    0.8529673218727112     │
│     tst/JaccardIndex3     │    0.9219774007797241     │
│     tst/JaccardIndex4     │    0.8663285374641418     │
│     tst/JaccardIndex5     │    0.3892577886581421     │
│     tst/JaccardIndex6     │    0.8720417618751526     │
│     tst/JaccardIndex7     │     0.652920126914978     │
│     tst/JaccardIndex8     │    0.5612532496452332     │
│         tst/Loss          │    2.0002822875976562     │
│     tst/MacroAccuracy     │    0.8498740792274475     │
│     tst/MacroF1Score      │    0.8419457077980042     │
│   tst/MacroJaccardIndex   │    0.7473906874656677     │
│    tst/MacroPrecision     │    0.8368901014328003     │
│      tst/MacroRecall      │    0.8498740792274475     │
│     tst/MicroAccuracy     │    0.9112805128097534     │
│     tst/MicroF1Score      │    0.9112805128097534     │
│   tst/MicroJaccardIndex   │    0.8371031880378723     │
│    tst/MicroPrecision     │    0.9112805128097534     │
│      tst/MicroRecall      │    0.9112805128097534     │
│      tst/Precision0       │            0.0            │
│      tst/Precision1       │    0.9048970341682434     │
│      tst/Precision2       │    0.9621855616569519     │
│      tst/Precision3       │    0.9363812804222107     │
│      tst/Precision4       │    0.9359167814254761     │
│      tst/Precision5       │    0.5756558775901794     │
│      tst/Precision6       │    0.8882149457931519     │
│      tst/Precision7       │     0.825587809085846     │
│      tst/Precision8       │    0.6662811040878296     │
│        tst/Recall0        │            0.0            │
│        tst/Recall1        │    0.9483305215835571     │
│        tst/Recall2        │    0.8825523853302002     │
│        tst/Recall3        │    0.9835895299911499     │
│        tst/Recall4        │     0.920958399772644     │
│        tst/Recall5        │    0.5458983778953552     │
│        tst/Recall6        │    0.9795466065406799     │
│        tst/Recall7        │     0.757390558719635     │
│        tst/Recall8        │    0.7807261347770691     │
└───────────────────────────┴───────────────────────────┘

[{'tst/Loss': 2.0002822875976562,
  'tst/MacroAccuracy': 0.8498740792274475,
  'tst/MacroF1Score': 0.8419457077980042,
  'tst/MacroJaccardIndex': 0.7473906874656677,
  'tst/MacroPrecision': 0.8368901014328003,
  'tst/MacroRecall': 0.8498740792274475,
  'tst/MicroAccuracy': 0.9112805128097534,
  'tst/MicroF1Score': 0.9112805128097534,
  'tst/MicroJaccardIndex': 0.8371031880378723,
  'tst/MicroPrecision': 0.9112805128097534,
  'tst/MicroRecall': 0.9112805128097534,
  'tst/Accuracy0': 0.0,
  'tst/Accuracy1': 0.9483305215835571,
  'tst/Accuracy2': 0.8825523853302002,
  'tst/Accuracy3': 0.9835895299911499,
  'tst/Accuracy4': 0.920958399772644,
  'tst/Accuracy5': 0.5458983778953552,
  'tst/Accuracy6': 0.9795466065406799,
  'tst/Accuracy7': 0.757390558719635,
  'tst/Accuracy8': 0.7807261347770691,
  'tst/F1Score0': 0.0,
  'tst/F1Score1': 0.9261047840118408,
  'tst/F1Score2': 0.9206501841545105,
  'tst/F1Score3': 0.9594050049781799,
  'tst/F1Score4': 0.9283773303031921,
  'tst/F1Score5': 0.560

In [4]:
# Qualitative Performance Evaluation
map_dirpath = os.path.join(log_dirpath, "qualitative")
os.makedirs(map_dirpath, exist_ok=True)

tile_id = "9-284-556"

TiledInferenceEngine(
    checkpoint_path=best_path,
    map_location="cpu",
    tile_store=BAG3DTileStore(),
    model=model,
).run(
    tile_id=tile_id,
    dst_filepath=os.path.join(map_dirpath, f"{tile_id}.map.pixel.best.tif"),
    params=TiledInferenceParameters(h_crop=512, h_stride=256, w_crop=512, w_stride=256),
)

Seed set to 0
C:\Documents\RoofSense\roofsense\utilities\file.py:70: UserWarning: The specified path: 'C:\\Documents\\RoofSense\\logs\\3dgeoinfo\\test\\qualitative\\9-284-556.map.pixel.last.tif' does not exist in the system. The value of the 'overwrite' flag will be ignored.
  warnings.warn(msg, UserWarning)
100%|██████████| 200/200 [13:00<00:00,  3.90s/it]
